In [1]:
from tsfresh_feature_extraction import *

In [2]:
set_seed(612)

In [3]:
train_extracted_minmax_large=np.load("./numpy/train_extracted_minmax_train_0322.npy")
test_extracted_minmax_large=np.load("./numpy/test_extracted_minmax_train_0322.npy")
device=CFG.device
CFG.epoches=350
longlist=np.concatenate((train_extracted_minmax_large,test_extracted_minmax_large),axis=\
                        0)[:,np.random.permutation(100)]
train_extracted_minmax_large,test_extracted_minmax_large=np.vsplit(longlist,np.array([24020]))
device=CFG.device
train_dataset_expend_encoded = AircraftDataset_expend_feature_extraction(df_train, train_label, 
                                                                         torch.FloatTensor(train_extracted_minmax_large),False)
train_encoded_loader = DataLoader(
    train_dataset_expend_encoded,
    batch_size=CFG.batch_size,
    shuffle=True,
    drop_last=True,
)
test_dataset = AircraftDataset_no_expend_feature_extraction(df_test, test_label, torch.FloatTensor(test_extracted_minmax_large))
test_encoded_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False)
for train_data in train_encoded_loader:None
for test_data in test_encoded_loader:None

In [218]:
path = DATA_PATH+'RUL.csv'
RUL_frame = pd.read_csv(path, header=None)
RUL = RUL_frame.values[:, 0]
RUL_y=pd.Series(RUL)
RUL_y.index+=1
aml_model_2=AML_model2(100,100,100,128,2,20,shared=False, attention=True,expand=False).to(device)
aml_model_2.load_state_dict(torch.load("./model_checkpoints_at/model_his_aml.pt"))
lifetime_list=[]
for i in range(len(test_dataset)):
    lifetime=test_dataset[i]["lifetime"].item()
    lifetime_list.append(lifetime)
lifetime_list=np.array(lifetime_list)

In [272]:
h_idx=35

In [273]:
aml_model_2(test_dataset[h_idx]["input"].unsqueeze(0).to(device))[0].item(),RUL[h_idx]

(187.09219360351562, 137)

In [274]:
same_lifetime_list=[]
for idx in range(len(train_dataset_expend_encoded)):
    if train_dataset_expend_encoded[idx]["lifetime"]==test_dataset[h_idx]["lifetime"]:
        same_lifetime_list.append(idx)
np.array(same_lifetime_list)       
    

array([   83,   335,   581,   796,  1061,  1267,  1538,  1955,  2215,
        2614,  3088,  3278,  3441,  3619,  3819,  4014,  4351,  4656,
        5096,  5318,  5649,  5862,  6047,  6236,  6723,  6909,  7145,
        7458,  7651,  7867,  8078,  8254,  8436,  8660,  9112,  9336,
        9653,  9970, 10164, 10445, 10626, 10914, 11100, 11414, 11587,
       11785, 11982, 12244, 12411, 12660, 12814, 12997, 13212, 13369,
       13556, 14074, 14262, 14470, 14641, 14933, 15116, 15308, 15547,
       15771, 15959, 16186, 16344, 16600, 16794, 16957, 17122, 17524,
       17749, 17957, 18143, 18395, 18541, 18789, 19003, 19162, 19302,
       19642, 19920, 20094, 20313, 20572, 20906, 21071, 21386, 21586,
       21760, 21909, 22060, 22224, 22609, 22768, 23252, 23520, 23820,
       23958])

In [275]:
distance_list=torch.zeros(len(same_lifetime_list))
y = test_dataset[h_idx]["input"]
for i,idx in enumerate(same_lifetime_list):
    x = train_dataset_expend_encoded[idx]["input"]
    distance = -F.pairwise_distance(x, y, p=2)
    #distance_list.append(distance)
    distance_list[i] = distance
distance_list

tensor([-4.5657, -2.0608, -4.9859, -3.4402, -4.5620, -4.4340, -2.6926, -4.6013,
        -3.3893, -4.3337, -1.7642, -5.4873, -4.4153, -5.4969, -4.3527, -1.4254,
        -2.1073, -2.2547, -1.8794, -2.2199, -1.4310, -5.9043, -5.2125, -3.1409,
        -4.9763, -5.0032, -4.4186, -5.3047, -4.7796, -4.6427, -5.2766, -4.3834,
        -1.9694, -4.0516, -4.2895, -5.2157, -1.9965, -2.1038, -1.7446, -5.0968,
        -3.1476, -2.4793, -2.4108, -5.3822, -1.5443, -1.7715, -4.6098, -5.1769,
        -2.0326, -5.5275, -4.7258, -4.9977, -5.3140, -4.6902, -4.5378, -5.6445,
        -1.8676, -5.3556, -1.7863, -1.3295, -3.7791, -1.8245, -5.2134, -4.4358,
        -4.8951, -5.3917, -4.5130, -4.4699, -4.8607, -5.1291, -2.6491, -1.9059,
        -3.2411, -5.3217, -2.2326, -5.0996, -2.0219, -5.0066, -5.1255, -5.4081,
        -1.7448, -1.5790, -4.8703, -1.3484, -2.1227, -4.3779, -5.0284, -1.5829,
        -2.4269, -5.4092, -5.5564, -5.4799, -5.5662, -2.3677, -5.6774, -3.5487,
        -1.3825, -1.6114, -5.4148, -5.66

In [276]:
diag=torch.exp(distance_list).unsqueeze(-1)
#diag=distance_list.unsqueeze(-1)
# diag[torch.where(diag < torch.quantile(diag, 0.1, dim=0,
#                                                 keepdim=False,
#                                                interpolation='nearest'))] = 0
diag_reg=diag / torch.sum(diag)
test_44=torch.sum(torch.FloatTensor(train_extracted_minmax_large[same_lifetime_list])*diag_reg,dim=0)

In [277]:
aml_model_2(test_44.unsqueeze(0).to(device))[0].item(),RUL[h_idx]

(225.58047485351562, 137)

# 特征增强函数

In [284]:
def feature_enhance2(instance_train,instance_test):
    #instance_test_enhanced = instance_test
    feature_enhanced_list=[]
    for test_idx in range(len(instance_test)):
        y = instance_test[test_idx]["input"]
        if instance_test[test_idx]["lifetime"]<100:
            same_lifetime_list=[]
            for idx in range(len(instance_train)):
                if instance_train[idx]["lifetime"]==instance_test[test_idx]["lifetime"]:
                    same_lifetime_list.append(idx)
            distance_list=torch.zeros(len(same_lifetime_list))

            for i,train_idx in enumerate(same_lifetime_list):
                x = instance_train[train_idx]["input"]
                distance = F.pairwise_distance(x, y, p=2)
                distance_list[i] = 1/distance
            #diag=torch.exp(distance_list).unsqueeze(-1)
            diag=distance_list.unsqueeze(-1)
#             diag[torch.where(diag < torch.quantile(diag, 0.5, dim=0,
#                                     keepdim=False,
#                                    interpolation='nearest'))] = 0
            diag_reg=diag / torch.sum(diag)
            feature_enhanced_list.append(torch.sum(instance_train.all_feature_list[same_lifetime_list]*diag_reg,
                                                   dim=0))
        
#         diag[torch.where(diag < torch.quantile(diag, 0.5, dim=0,
#                                             keepdim=False,
#                                            interpolation='nearest'))] = 0
#         diag_reg=diag / torch.sum(diag)
        #instance_test_enhanced[test_idx]["input"] = torch.sum(instance_train.all_feature_list*diag_reg,dim=0)

        else:
            feature_enhanced_list.append(y)
    return feature_enhanced_list     

In [285]:
feature_enhanced_list = feature_enhance2(train_dataset_expend_encoded,test_dataset)

In [286]:
test_dataset_enhanced = AircraftDataset_no_expend_feature_extraction(df_test, 
                                                            test_label, 
                                                            feature_enhanced_list)
test_encoded_enhanced_loader = DataLoader(
    test_dataset_enhanced,
    batch_size=1,
    shuffle=False)
pred_2=[]
for idx,test_data in enumerate(test_encoded_enhanced_loader):
    pred_2_q=aml_model_2(test_data["input"].to(device))[0].item()
    pred_2.append(pred_2_q)
#RUL
path = DATA_PATH+'RUL.csv'
RUL_frame = pd.read_csv(path, header=None)
RUL = RUL_frame.values[:, 0]
np.sum(np.ceil((abs(np.array(pred_2)-RUL))))#目测结果

1921.0

In [287]:
np.where(np.ceil((abs(np.array(pred_2)-RUL)))>80)

(array([14], dtype=int64),)

In [288]:
lifetime_list[np.where(np.ceil((abs(np.array(pred_2)-RUL)))>50)]

array([112, 133,  38,  50, 162,  78, 105,  68, 218,  56])

In [289]:
get_Accuracy(np.array(pred_2), RUL, lifetime_list)

matrix([[2.50000000e+01, 1.53524766e-02, 4.10804652e-03],
        [5.00000000e+01, 1.70854534e-02, 3.62896903e-03],
        [7.50000000e+01, 5.24026253e-02, 1.06849537e-02],
        [1.00000000e+02, 7.53767752e-02, 1.25226407e-02],
        [1.25000000e+02, 8.50634632e-02, 1.18799379e-02],
        [1.50000000e+02, 8.84322751e-02, 1.05493540e-02]])

# 特征增强函数2

In [278]:
h_idx=35

In [279]:
aml_model_2(test_dataset[h_idx]["input"].unsqueeze(0).to(device))[0].item(),RUL[h_idx]

(187.09219360351562, 137)

In [280]:
same_lifetime_list=[]
for idx in range(len(train_dataset_expend_encoded)):
    if train_dataset_expend_encoded[idx]["lifetime"]==test_dataset[h_idx]["lifetime"]:
        same_lifetime_list.append(idx)
np.array(same_lifetime_list)   

array([   83,   335,   581,   796,  1061,  1267,  1538,  1955,  2215,
        2614,  3088,  3278,  3441,  3619,  3819,  4014,  4351,  4656,
        5096,  5318,  5649,  5862,  6047,  6236,  6723,  6909,  7145,
        7458,  7651,  7867,  8078,  8254,  8436,  8660,  9112,  9336,
        9653,  9970, 10164, 10445, 10626, 10914, 11100, 11414, 11587,
       11785, 11982, 12244, 12411, 12660, 12814, 12997, 13212, 13369,
       13556, 14074, 14262, 14470, 14641, 14933, 15116, 15308, 15547,
       15771, 15959, 16186, 16344, 16600, 16794, 16957, 17122, 17524,
       17749, 17957, 18143, 18395, 18541, 18789, 19003, 19162, 19302,
       19642, 19920, 20094, 20313, 20572, 20906, 21071, 21386, 21586,
       21760, 21909, 22060, 22224, 22609, 22768, 23252, 23520, 23820,
       23958])

In [281]:
y0 = torch.zeros(100)
for sensor in range(100):
    feature_train = torch.FloatTensor(train_extracted_minmax_large[same_lifetime_list])[:,sensor]
    distance_list=torch.zeros(len(same_lifetime_list))
    y = test_dataset[h_idx]["input"][sensor]
    for i,idx in enumerate(same_lifetime_list):
        x = train_dataset_expend_encoded[idx]["input"][sensor]
        #distance = -F.pairwise_distance(x, y, p=2)
        distance = F.pairwise_distance(x, y, p=2)
        #distance_list.append(distance)
        distance_list[i] = distance
    #diag=torch.exp(distance_list)
    diag=distance_list
#         diag[torch.where(diag < torch.quantile(diag, 0.6, dim=0,
#                                             keepdim=False,
#                                            interpolation='nearest'))] = 0
    diag_reg=diag / torch.sum(diag)
    y0[sensor] = torch.inner(feature_train,diag_reg)

In [282]:
y0

tensor([-0.0201, -0.0640,  0.0615, -0.3585, -0.3828, -0.0188,  0.1647, -0.2367,
         0.1105,  0.1549,  0.1823,  0.0810,  0.3176, -0.4956, -0.6453,  0.2138,
        -0.4565,  0.2203,  0.0882,  0.1454,  0.0912,  0.1308,  0.4708,  0.4526,
        -0.4412,  0.1986,  0.2967, -0.2666,  0.4051, -0.7785,  0.0959, -0.0189,
        -0.3461, -1.0000, -0.6804,  0.1461,  0.4184, -1.0000, -0.1342,  0.2994,
         0.0494, -0.3461, -0.8582, -0.5291, -0.1281,  0.2386,  0.2896, -0.1071,
        -0.3356,  0.4243, -0.6759,  0.2387, -0.2464,  0.4334, -0.3439,  0.1849,
        -0.3089, -0.6752,  0.1823, -0.3239,  0.2325,  0.0422,  0.1896, -0.2940,
         0.1434, -0.0189, -0.6861, -0.1281, -0.2036, -0.2626, -0.3722, -0.6384,
        -0.4059, -0.4618,  0.3231, -0.8641,  0.4618, -0.3743,  0.4414, -0.6817,
         0.1701,  0.1626,  0.0659,  0.1550, -0.4843,  0.1387, -0.0195, -0.6078,
         0.2405, -0.0283, -0.2854,  0.2620,  0.4723,  0.1740,  0.2646,  0.1021,
         0.2460,  0.4476,  0.1543,  0.28

In [283]:
aml_model_2(y0.unsqueeze(0).to(device))[0].item(),RUL[h_idx]

(91.31954193115234, 137)

In [262]:
def feature_enhance3(instance_train,instance_test):
    #instance_test_enhanced = instance_test
    feature_enhanced_list=[]
    for test_idx in range(len(instance_test)):
        y = instance_test[test_idx]["input"]
        if instance_test[test_idx]["lifetime"]<100:
            same_lifetime_list=[]
            for idx in range(len(instance_train)):
                if instance_train[idx]["lifetime"]==instance_test[test_idx]["lifetime"]:
                    same_lifetime_list.append(idx)
            distance_list=torch.zeros(len(same_lifetime_list))

            for i,train_idx in enumerate(same_lifetime_list):
                x = instance_train[train_idx]["input"]
                distance = -F.pairwise_distance(x, y, p=2)
                distance_list[i] = distance
            diag=torch.exp(distance_list).unsqueeze(-1)
            diag[torch.where(diag < torch.quantile(diag, 0.5, dim=0,
                                    keepdim=False,
                                   interpolation='nearest'))] = 0
            diag_reg=diag / torch.sum(diag)
            feature_enhanced_list.append(torch.sum(instance_train.all_feature_list[same_lifetime_list]*diag_reg,
                                                   dim=0))
        
#         diag[torch.where(diag < torch.quantile(diag, 0.5, dim=0,
#                                             keepdim=False,
#                                            interpolation='nearest'))] = 0
#         diag_reg=diag / torch.sum(diag)
        #instance_test_enhanced[test_idx]["input"] = torch.sum(instance_train.all_feature_list*diag_reg,dim=0)

        else:
            feature_enhanced_list.append(y)
    return feature_enhanced_list     